In [4]:
import sqlite3

from sklearn.metrics import r2_score
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
from scipy.stats import fisk

In [12]:
db_path = "F1_timingdata_2014_2019 copy 2.sqlite"
data_loader = DataLoader(db_path=db_path)
dataframes = data_loader.load_data()

In [26]:
import pandas as pd
import numpy as np

def generate_pit_stop_strategy(season: int, location: str, dataframes: dict):
    """
    Generates pit stop strategies for each driver in a given season and location.
    
    Args:
        season (int): The racing season.
        location (str): Grand Prix location.
        dataframes (dict): Dictionary containing race-related data.
    
    Returns:
        dict: Pit stop strategies for each driver.
    """
    # Load required data from the dataframes
    df_pit_stops = dataframes.get("laps", pd.DataFrame())
    df_races = dataframes.get("races", pd.DataFrame())
    df_drivers = dataframes.get("drivers", pd.DataFrame())
    
    # Find the race ID for the given season and location
    race_row = df_races[(df_races["season"] == season) & (df_races["location"] == location)]
    if race_row.empty:
        raise ValueError(f"No race found for location '{location}' in season {season}.")
    
    race_id = race_row.iloc[0]["id"]
    
    # Filter pit stop data for the selected race
    pit_stop_data = df_pit_stops[df_pit_stops["race_id"] == race_id]
    
    # Extract unique drivers from the filtered data
    drivers = pit_stop_data["driver_id"].unique()
    
    pit_stop_strategies = {}
    
    for driver_id in drivers:
        driver_pit_stops = pit_stop_data[pit_stop_data["driver_id"] == driver_id]
        driver_name = df_drivers[df_drivers["id"] == driver_id]["name"].values[0]
        
        # Find the starting tire compound and tire age (lap 0)
        starting_tire = driver_pit_stops[driver_pit_stops["lapno"] == 0]
        if not starting_tire.empty:
            starting_compound = starting_tire.iloc[0]["compound"]
            starting_tire_age = starting_tire.iloc[0]["tireage"]
        else:
            starting_compound = "Unknown"
            starting_tire_age = "Unknown"
        
        strategy = {"starting_compound": starting_compound, "starting_tire_age": starting_tire_age}
        
        # Filter only actual pit stops (where pitstopduration is not NaN or zero)
        valid_pit_stops = driver_pit_stops[driver_pit_stops["pitstopduration"].notna() & (driver_pit_stops["pitstopduration"] > 0)]
        
        for idx, (lap, duration, compound, tireage) in enumerate(
            zip(valid_pit_stops["lapno"], valid_pit_stops["pitstopduration"], valid_pit_stops["compound"], valid_pit_stops["tireage"])
        ):
            pitstop_interval = [lap,lap]
            strategy[idx+1] = {
                "compound": compound,
                "pitstop_interval": pitstop_interval,
                "pit_stop_lap": lap,
                "tire_age": tireage
            }
        
        pit_stop_strategies[driver_name] = strategy
    
    return pit_stop_strategies

generate_pit_stop_strategy(season=2016, location="Austin",dataframes=dataframes)


{'Lewis Hamilton': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tire_age': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tire_age': 20}},
 'Nico Rosberg': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tire_age': 12},
  2: {'compound': 'A2',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tire_age': 21}},
 'Daniel Ricciardo': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [8, 8],
   'pit_stop_lap': 8,
   'tire_age': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [25, 25],
   'pit_stop_lap': 25,
   'tire_age': 17}},
 'Max Verstappen': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [9, 9],
   'pit_stop_lap': 9,
   'tire_age': 11},
  2: {'comp

In [ ]:
{'Lewis Hamilton': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tireage': 20}},
 'Nico Rosberg': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 12},
  2: {'compound': 'A2',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tireage': 21}},
 'Daniel Ricciardo': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [8, 8],
   'pit_stop_lap': 8,
   'tireage': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [25, 25],
   'pit_stop_lap': 25,
   'tireage': 17}},
 'Max Verstappen': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [9, 9],
   'pit_stop_lap': 9,
   'tireage': 11},
  2: {'compound': 'A3',
   'pitstop_interval': [26, 26],
   'pit_stop_lap': 26,
   'tireage': 17}},
 'Kimi Raikkonen': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [8, 8],
   'pit_stop_lap': 8,
   'tireage': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [24, 24],
   'pit_stop_lap': 24,
   'tireage': 16},
  3: {'compound': 'A4',
   'pitstop_interval': [38, 38],
   'pit_stop_lap': 38,
   'tireage': 14}},
 'Sebastian Vettel': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [14, 14],
   'pit_stop_lap': 14,
   'tireage': 16},
  2: {'compound': 'A3',
   'pitstop_interval': [29, 29],
   'pit_stop_lap': 29,
   'tireage': 15},
  3: {'compound': 'A2',
   'pitstop_interval': [53, 53],
   'pit_stop_lap': 53,
   'tireage': 24}},
 'Nico Hulkenberg': {'starting_compound': 'A4', 'starting_tire_age': 2},
 'Valtteri Bottas': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [1, 1],
   'pit_stop_lap': 1,
   'tireage': 3},
  2: {'compound': 'A3',
   'pitstop_interval': [20, 20],
   'pit_stop_lap': 20,
   'tireage': 19}},
 'Felipe Massa': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [29, 29],
   'pit_stop_lap': 29,
   'tireage': 18},
  3: {'compound': 'A2',
   'pitstop_interval': [54, 54],
   'pit_stop_lap': 54,
   'tireage': 25}},
 'Carlos Sainz Jnr': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [30, 30],
   'pit_stop_lap': 30,
   'tireage': 19}},
 'Sergio Perez': {'starting_compound': 'A4',
  'starting_tire_age': 0,
  1: {'compound': 'A4',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 10},
  2: {'compound': 'A2',
   'pitstop_interval': [27, 27],
   'pit_stop_lap': 27,
   'tireage': 17}},
 'Fernando Alonso': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 11},
  2: {'compound': 'A2',
   'pitstop_interval': [30, 30],
   'pit_stop_lap': 30,
   'tireage': 19}},
 'Daniil Kvyat': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [21, 21],
   'pit_stop_lap': 21,
   'tireage': 21}},
 'Esteban Gutierrez': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [13, 13],
   'pit_stop_lap': 13,
   'tireage': 13}},
 'Jolyon Palmer': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [15, 15],
   'pit_stop_lap': 15,
   'tireage': 15},
  2: {'compound': 'A3',
   'pitstop_interval': [26, 26],
   'pit_stop_lap': 26,
   'tireage': 11}},
 'Marcus Ericsson': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [17, 17],
   'pit_stop_lap': 17,
   'tireage': 17}},
 'Romain Grosjean': {'starting_compound': 'A4',
  'starting_tire_age': 0,
  1: {'compound': 'A4',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [27, 27],
   'pit_stop_lap': 27,
   'tireage': 17}},
 'Kevin Magnussen': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [13, 13],
   'pit_stop_lap': 13,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [27, 27],
   'pit_stop_lap': 27,
   'tireage': 14},
  3: {'compound': 'A2',
   'pitstop_interval': [43, 43],
   'pit_stop_lap': 43,
   'tireage': 16}},
 'Jenson Button': {'starting_compound': 'A4',
  'starting_tire_age': 0,
  1: {'compound': 'A4',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 10},
  2: {'compound': 'A2',
   'pitstop_interval': [28, 28],
   'pit_stop_lap': 28,
   'tireage': 18}},
 'Pascal Wehrlein': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [13, 13],
   'pit_stop_lap': 13,
   'tireage': 13},
  2: {'compound': 'A2',
   'pitstop_interval': [30, 30],
   'pit_stop_lap': 30,
   'tireage': 17}},
 'Felipe Nasr': {'starting_compound': 'A2',
  'starting_tire_age': 0,
  1: {'compound': 'A2',
   'pitstop_interval': [29, 29],
   'pit_stop_lap': 29,
   'tireage': 29}},
 'Esteban Ocon': {'starting_compound': 'A2',
  'starting_tire_age': 0,
  1: {'compound': 'A2',
   'pitstop_interval': [17, 17],
   'pit_stop_lap': 17,
   'tireage': 17},
  2: {'compound': 'A3',
   'pitstop_interval': [26, 26],
   'pit_stop_lap': 26,
   'tireage': 9},
  3: {'compound': 'A3',
   'pitstop_interval': [44, 44],
   'pit_stop_lap': 44,
   'tireage': 18}}}

In [18]:
dataframes["laps"]

,race_id,lapno,position,driver_id,laptime,racetime,gap,interval,compound,tireage,pitintime,pitstopduration,nextcompound,startlapprog_vsc,endlapprog_vsc,age_vsc,startlapprog_sc,endlapprog_sc,age_sc
0,1,0,1,1,NaN,0.000,NaN,NaN,A3,2,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,2,2,NaN,0.000,NaN,NaN,A3,2,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0,3,3,NaN,0.000,NaN,NaN,A3,2,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0,4,4,NaN,0.000,NaN,NaN,A3,2,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0,5,5,NaN,0.000,NaN,NaN,A3,2,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134001,121,55,2,27,101.119,5662.487,16.772,16.772,A4,30,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
134002,121,55,3,40,102.829,5689.150,43.435,26.663,A7,17,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
134003,121,55,4,15,102.180,5690.094,44.379,0.944,A4,26,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
134004,121,55,5,12,100.128,5710.072,64.357,19.978,A6,17,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
class Overtaking():
    pass

In [18]:
class PitStop():
    def __init__(self, team, gp_location, season, dataframes):
        self.team = team.name
        self.gp_location = gp_location
        self.len_train_df = 2
        
        self.season = season
        self.dfs = dataframes
        
        # Check if we still need it, because we will get it through Run class
        # Retrieve the race_id for the given season and location
        df_races = self.dfs["races"]
        race_row = df_races[
            (df_races["season"] == self.season) & (df_races["location"] == self.gp_location)
        ]

        
        if race_row.empty:
            raise ValueError(f"No race found for location '{self.gp_location}' in season {self.season}.")
        self.race_id = race_row["id"].iloc[0]

    def calculate_best_pit_stop_duration(self):
        df_laps=self.dfs["laps"]
        df_races = self.dfs['races']
        location = df_races[df_races['id']==self.race_id]['location'].iloc[0]
        seasons_to_train = [self.season - x for x in range (1, self.len_train_df +1)]
        races_to_train = list(df_races[(df_races['location']==location) & (df_races['season'].isin(seasons_to_train))]['id'])

        min_pit_stop_duration_per_race = df_laps[df_laps['race_id'].isin(races_to_train)].dropna(subset=['pitstopduration']).groupby(['race_id'])[['pitstopduration']].quantile(q=0.025)
        avg_min_pit_stop_duration = min_pit_stop_duration_per_race['pitstopduration'].mean()
        self.avg_min_pit_stop_duration = avg_min_pit_stop_duration

    def calibrate_pit_stop_variability_law(self):
        df_laps=self.dfs["laps"]
        df_staterfields = self.dfs["starterfields"]
        df_races = self.dfs["races"]

        df_laps_with_season = df_laps.merge(
            df_races[["id", "season"]], 
            left_on="race_id",
            right_on="id",
            how="left"
        ).drop(columns=["id"])  # We can drop the duplicated "id" column
        
        df_merged = df_laps_with_season.merge(
            df_staterfields[["race_id", "driver_id", "team"]],
            on=["race_id", "driver_id"],
            how="left"
        )
        
        df = df_merged[
            (df_merged['team'] == self.team) & 
            (df_merged['season'] == self.season) &
            (df_merged['race_id'] < self.race_id) &
            (df_merged['pitstopduration'].notna()) & 
            (df_merged['pitstopduration'] < 700)
        ].copy()
        
        df[["pitstop_diff"]]=df[['pitstopduration']]- self.avg_min_pit_stop_duration
        shape, loc, scale=fisk.fit(df["pitstop_diff"])
        return [shape, loc, scale]

    def calculate_pit_stop_duration(self):
        shape, loc, scale = self.calibrate_pit_stop_variability_law()
        variability = fisk.rvs(shape, loc=loc, scale=scale, size=1, random_state=None)[0]
        pit_stop = self.avg_min_pit_stop_duration + variability
        return pit_stop
        


In [9]:
def main():
    db_path = "F1_timingdata_2014_2019.sqlite"
    data_loader = DataLoader(db_path=db_path)
    dataframes = data_loader.load_data()
    mercedes=Team("Mercedes")
    pit_stop = PitStop(
        team=mercedes, 
        gp_location="SaoPaulo",
        season=2016, 
        dataframes=dataframes
    )

    display(dataframes["starterfields"])
    pit_stop.calculate_best_pit_stop_duration()
    print("Average Min Pit Stop Duration:", pit_stop.avg_min_pit_stop_duration)

    calculated_duration = pit_stop.calculate_pit_stop_duration()
    print("Calculated Pit Stop Duration:", calculated_duration)


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"[ERROR] An exception occurred: {e}")


[ERROR] An exception occurred: Execution failed on sql 'SELECT * FROM drivers': no such table: drivers


In [2]:
class DataLoader:
    """
    Class responsible for loading data from an SQLite database.
    """

    def __init__(self, db_path: str):
        self.db_path = db_path
        self.dataframes = {}

    def load_data(self) -> dict:
        """
        Load data from the SQLite database and store it in a dictionary.

        Returns:
            dict: A dictionary where keys are table names and values are DataFrames.
        """
        connection = sqlite3.connect(self.db_path)
        tables = [
            "drivers",
            "fcyphases",
            "laps",
            "qualifyings",
            "races",
            "retirements",
            "starterfields",
        ]
        self.dataframes = {
            table: pd.read_sql_query(f"SELECT * FROM {table}", connection)
            for table in tables
        }
        connection.close()
        return self.dataframes



In [33]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

class FuelAndTireModel:
    """
    Classe refactorisée pour éviter les modifications destructives
    des DataFrames partagés et gérer explicitement les catégories de pneus.
    """

    # Définissez ici toutes les catégories possibles pour la colonne "compound"
    # selon votre jeu de données et vos besoins de simulation
    ALL_COMPOUNDS = ["A1", "A2", "A3", "A4", "A6", "A7",'I','W']  
    
    def __init__(self, season: int, driver_id: int, race_id: int, dataframes: dict):
        """
        Args:
            season (int): Saison à analyser.
            driver_id (int): ID du pilote.
            race_id (int): ID de la course ciblée pour le test.
            dataframes (dict): Dictionnaire de DataFrames (copié localement).
        """
        self.season = season
        self.driver_id = driver_id
        self.race_id = race_id
        
        # Copie locale pour ne pas modifier dataframes global
        self.dfs_local = {name: df.copy() for name, df in dataframes.items()}

        self.best_qualif_times = pd.DataFrame()
        
        # DataFrame local qui contiendra nos tours filtrés
        self.laps_df = pd.DataFrame()

        # Jeu d'entraînement / Jeu de test
        self.train_data = pd.DataFrame()
        self.test_data = pd.DataFrame()

        # Modèle OLS et sa volatilité
        self.model = None
        self.variability = None 

        # Stocke les prédictions éventuelles
        self.predictions = pd.Series(dtype=float)

    def clean_data(self):
        """
        Applique divers filtres (courses terminées, exclusion pluie, safety car, pit stops...)
        et stocke le résultat dans self.laps_df au lieu de modifier self.dfs_local["laps"].
        """
        laps_df = self.dfs_local["laps"]
        races_df = self.dfs_local["races"]
        starterfields_df = self.dfs_local["starterfields"]
        fcyphases_df = self.dfs_local["fcyphases"]

        # Filtrer les tours du pilote pour la saison
        race_ids_season = races_df[races_df["season"] == self.season]["id"]
        laps_df = laps_df[
            (laps_df["driver_id"] == self.driver_id)
            & (laps_df["race_id"].isin(race_ids_season))
        ]

        # Conserver seulement les courses terminées par le pilote
        starterfields_df = starterfields_df[
            (starterfields_df["driver_id"] == self.driver_id)
            & (starterfields_df["race_id"].isin(race_ids_season))
        ]
        finished_races = starterfields_df[starterfields_df["status"] == "F"]
        laps_df = laps_df[laps_df["race_id"].isin(finished_races["race_id"])]

        # Exclure les courses pluvieuses (ajustez selon vos besoins)
        rainy_races = ["Budapest", "SaoPaulo"]
        rainy_race_ids = races_df[races_df["location"].isin(rainy_races)]["id"]
        laps_df = laps_df[~laps_df["race_id"].isin(rainy_race_ids)]

        # Exclure les tours sous Safety Car ou Virtual Safety Car
        fcyphases_df = fcyphases_df[fcyphases_df["race_id"].isin(race_ids_season)]
        for _, row in fcyphases_df.iterrows():
            begin = row["startlap"]
            end = row["endlap"]
            laps_df = laps_df[
                ~(
                    (laps_df["race_id"] == row["race_id"]) 
                    & (laps_df["lapno"].between(begin, end))
                )
            ]

        # Exclure les tours avec pit stop et le tour suivant
        laps_df = laps_df.reset_index(drop=True)
        pit_in_index = laps_df[~laps_df["pitintime"].isna()].index
        pit_out_index = [i + 1 for i in pit_in_index]
        laps_df = laps_df[~laps_df.index.isin(list(pit_in_index) + pit_out_index)]

        # Stockage dans self.laps_df
        self.laps_df = laps_df

    def get_best_qualif_time(self):
        """
        Calcule le meilleur temps de qualif (min de Q1, Q2, Q3) pour chaque course,
        puis fait un merge sur self.laps_df.
        """
        # Vérification
        if self.laps_df.empty:
            raise RuntimeError("self.laps_df is empty. Call clean_data() before get_best_qualif_time().")

        qualif_laps_df = self.dfs_local["qualifyings"]
        valid_race_ids = self.laps_df["race_id"].unique()

        # Restreindre aux qualifs du pilote + courses valides
        qualif_laps_df = qualif_laps_df[
            (qualif_laps_df["driver_id"] == self.driver_id)
            & (qualif_laps_df["race_id"].isin(valid_race_ids))
        ]

        # Calcul du meilleur temps parmi Q1, Q2, Q3
        best_qualif_times = (
            qualif_laps_df[["race_id", "q1laptime", "q2laptime", "q3laptime"]]
            .groupby("race_id")
            .min()
            .min(axis=1)
        )

        self.best_qualif_times = pd.DataFrame(best_qualif_times, columns=["best_qualif_time"])

        # Merge local
        self.laps_df = self.laps_df.merge(
            self.best_qualif_times, on="race_id", how="left"
        )

    def add_features(self):
        """
        Ajoute notamment le fuel (estimation simpliste) et éventuellement d'autres features.
        """
        if self.laps_df.empty:
            raise RuntimeError("self.laps_df is empty. Call clean_data() before add_features().")

        laps_df = self.laps_df.copy()

        # Exemple d'estimation du fuel
        laps_df["fuelc"] = (
            100
            - (100 / laps_df.groupby("race_id")["lapno"].transform("max"))
            * laps_df["lapno"]
        )

        self.laps_df = laps_df

    def clean_to_regression(self):
        """
        Prépare la colonne 'corrected_lap_time' et gère la variable catégorielle 'compound'.
        """
        if self.laps_df.empty:
            raise RuntimeError("self.laps_df is empty. Call previous steps first.")

        required_features = ["laptime", "best_qualif_time", "fuelc", "compound", "tireage"]

        # Corrected Lap Time
        self.laps_df["corrected_lap_time"] = (
            self.laps_df["laptime"] - self.laps_df["best_qualif_time"]
        )

        # On déclare explicitement la liste de catégories pour 'compound'
        self.laps_df["compound"] = pd.Categorical(
            self.laps_df["compound"],
            categories=self.ALL_COMPOUNDS
        )

        # Exclure les lignes incomplètes
        self.laps_df.dropna(subset=required_features, inplace=True)

    def split_train_test(self):
        """
        Sépare en ensemble d'entraînement (races [start_race_id, race_id - 1])
        et ensemble de test (race_id).
        """
        if self.laps_df.empty:
            raise RuntimeError("self.laps_df is empty. Make sure you called previous steps.")

        start_race_id = np.min(self.laps_df["race_id"])

        self.train_data = self.laps_df[
            self.laps_df["race_id"].between(start_race_id, self.race_id - 1)
        ]
        self.test_data = self.laps_df[self.laps_df["race_id"] == self.race_id]

    def regression(self):
        """
        Entraîne un modèle OLS sur train_data avec la formule indiquée.
        """
        formula = "corrected_lap_time ~ fuelc + C(compound) + C(compound):tireage"
        self.model = smf.ols(formula=formula, data=self.train_data).fit()

    def get_model_and_variability(self):
        """
        Pipeline complet :
          - Nettoyage / merge des données
          - Ajout de features
          - Split train/test
          - Régression OLS
          - Calcul de la volatilité (std. dev) des résidus

        Returns:
            (model, variability): Le modèle entraîné et l'écart-type des résidus.
        """
        self.clean_data()
        self.get_best_qualif_time()
        self.add_features()
        self.clean_to_regression()
        self.split_train_test()
        self.regression()

        # Volatilité = écart-type des résidus
        self.variability = np.std(self.model.resid)

        return (self.model, self.variability)


In [5]:
class DNFModel():
    def get_parameters(self, driver_name: str, season: int):
        """
        Example method that returns probabilities for DNF.
        """
        return 0.2, 0.2  # (prob_accident, prob_failure)

In [6]:

class Team:
    def __init__(self, name: str):
        # Initialiser strategy
        # ET driveréé
        self.name = name


class TeamRegistry:
    """
    Registry (or cache) to ensure a single instance of Team per unique name.
    """
    _teams_cache = {}

    @classmethod
    def get_team(cls, name: str) -> Team:
        if name not in cls._teams_cache:
            cls._teams_cache[name] = Team(name)
        return cls._teams_cache[name]



In [43]:

class Driver:
    def __init__(self, season: int,race_id, dataframes: dict, name: str):
        self.season = season

        # Ajouter driver.best_qualif_time
        self.dataframes = dataframes
        self.name = name

        self.team = None
        self.driver_id = None
        self.initials = None

        # Simulation attributes
        self.position = None
        self.cumulative_lap_time = 0
        self.compound = "A2"
        self.tire_age = 0
        self.fuelc = 100
        self.next_pit_stop = 1

        # Strategy and DNF attributes
        self.pit_stops_info = {
            1: {
                "compound": "A2",
                "pitstop_interval": [10, 20],
                "pit_stop_lap": 15  # ou aléatoire entre les bornes de pitstop_interval
            },
            2: {
                "compound": "A2",
                "pitstop_interval": [25, 35],
                "pit_stop_lap": 30
            }
            # Et ainsi de suite pour les autres arrêts
        } # Modifer None

        self.accident_dnf_probability = None
        self.failure_dnf_probability = None
        self.accident_dnf_lap = None
        self.failure_dnf_lap = None
        self.earliest_dnf_lap = None
        self.alive = True

        # Fuel & Tire attributes
        self.fuel_tire_model = None
        self.variability = None

        self.get_driver_parameters(race_id)
        
    def get_driver_parameters(self, race_id):
        drivers_df = self.dataframes["drivers"]
        starterfields_df = self.dataframes["starterfields"]
        races_df = self.dataframes["races"]

        # Get driver ID and initials
        driver_row = drivers_df[drivers_df["name"] == self.name]
        if driver_row.empty:
            raise ValueError(f"Driver '{self.name}' not found in 'drivers' table.")

        self.driver_id = driver_row.iloc[0]["id"]
        self.initials = driver_row.iloc[0]["initials"]

        # Get team
        merged_data = starterfields_df.merge(
            races_df,
            left_on="race_id",
            right_on="id",
            suffixes=("_sf", "_races")
        )
        team_row = merged_data[
            (merged_data["driver_id"] == self.driver_id) &
            (merged_data["season"] == self.season)
        ]
        if not team_row.empty:
            self.team = TeamRegistry.get_team(team_row.iloc[0]["team"])

        # Get DNF probabilities
        dnf_model = DNFModel()
        self.accident_dnf_probability, self.failure_dnf_probability = dnf_model.get_parameters(
            driver_name=self.name,
            season=self.season
        )

        # Get fuel and tire parameters
        fuel_tire_model_object = FuelAndTireModel(season=self.season, driver_id=self.driver_id, race_id=race_id, dataframes=self.dataframes)
        self.fuel_tire_model, self.variability = fuel_tire_model_object.get_model_and_variability()

    def update_status(self, current_lap: int):
        if self.alive and self.earliest_dnf_lap == current_lap:
            self.alive = False

    def update_info(self, current_lap: int, total_laps: int):
        """
        Incrémente l'usure des pneus + met à jour le carburant.
        """
        self.tire_age += 1

       
        self.fuelc = 100 - (100.0 / total_laps) * current_lap


In [45]:
class Run:
    def __init__(self, season: int, gp_location: str, dataframes: dict):
        self.season = season
        self.gp_location = gp_location
        self.race_id = None
        self.dataframes = dataframes

        self.safety_car_laps=[12,27]
        self.number_of_laps = None
        self.drivers_list = []
        self.starting_grid = None

        # Ajout d'une colonne "status"
        self.laps_summary = pd.DataFrame(
            columns=[
                "lap", 
                "driver_id", 
                "position", 
                "lap_time", 
                "cumulative_lap_time",
                "status"
            ]
        )

        self._initialize_parameters()

    def run(self):
        # On va de 1 à number_of_laps (inclus)
        for lap in range(1, self.number_of_laps + 1):
            # Stocker le lap_time de chaque pilote dans un dict local
            # pour éviter la collision des variables
            lap_times = {}

            # 1) Mettre à jour le statut vivant ou DNF
            # 2) Calculer le lap_time pour ceux qui sont encore "alive"
            for driver in self.drivers_list:
                driver.update_status(lap)  # => peut passer driver.alive à False si lap == earliest_dnf_lap

                if driver.alive:
                    # Pilote en course => on calcule un temps
                    driver.update_info(lap, self.number_of_laps)  # Exemple: incrémenter tire_age etc.
                    lap_time = self.compute_lap_time(driver, lap)
                    pit_stop_time = self.pit_stop(driver, lap)
                    total_lap_time = lap_time + pit_stop_time

                    driver.cumulative_lap_time += total_lap_time
                    lap_times[driver.driver_id] = total_lap_time
                else:
                    # Pilote plus en course => on ne calcule plus de temps
                    lap_times[driver.driver_id] = np.nan  # ou 0, selon votre convention

            # Ensuite, on met à jour la position de ceux qui sont encore en piste
            # (les DNF n'ont plus de position "active")
            for driver in self.drivers_list:
                if driver.alive:
                    self.get_driver_position(driver) 
                else:
                    driver.position = np.nan

            # 3) Remplir laps_summary
            # Pour ce tour, on va ajouter **une ligne** pour chaque pilote
            # MAIS si un pilote vient de DNF à ce tour,
            # on l'ajoute ici avec "DNF", et les tours suivants on ne l'ajoutera plus.
            for driver in self.drivers_list:
                # Si on souhaite que la "dernière apparition" soit le tour de DNF
                # il faut filtrer : on ajoute une ligne seulement si driver.alive
                # OU s'il vient de DNF à ce tour précis (lap == earliest_dnf_lap).
                
                if driver.alive or (driver.earliest_dnf_lap == lap):
                    status_str = "running" if driver.alive else "DNF"

                    self.laps_summary = pd.concat([
                        self.laps_summary,
                        pd.DataFrame([{
                            "lap": lap,
                            "driver_id": driver.driver_id,
                            "position": driver.position,
                            "lap_time": lap_times[driver.driver_id],
                            "cumulative_lap_time": driver.cumulative_lap_time,
                            "status": status_str
                        }])
                    ], ignore_index=True)

                # Si un pilote est déjà DNF avant ce tour, on n'ajoute plus rien pour lui.

    def compute_lap_time(self, driver, current_lap):
        features = pd.DataFrame({
            "fuelc": [driver.fuelc],
            "compound": [driver.compound],
            "tireage": [driver.tire_age]
        })
        base_time = driver.fuel_tire_model.predict(features).iloc[0]  # Ne prend pas best_qualif  
        stochastic_term = np.random.normal(0, driver.variability)  # Mettre driver.best_qualif_time
        lap_time = base_time + stochastic_term

        # Safety car ?
        if current_lap in self.safety_car_laps:
            lap_time *= 1.2

        return lap_time

    def pit_stop(self, driver, current_lap):
        try:
            if driver.next_pit_stop in driver.pit_stops_info:
                pit_stop_data = driver.pit_stops_info[driver.next_pit_stop]
                is_pit_stop_lap = (current_lap == pit_stop_data["pit_stop_lap"])
                is_safety_car_pit_stop = (
                    current_lap in self.safety_car_laps and
                    current_lap in range(
                        pit_stop_data["pitstop_interval"][0],
                        pit_stop_data["pitstop_interval"][1] + 1
                    )
                )
                if is_pit_stop_lap or is_safety_car_pit_stop:
                    pit_stop = PitStop(
                        team=driver.team,
                        gp_location=self.gp_location,
                        season=self.season,
                        dataframes=self.dataframes
                    )
                    pit_stop.calculate_best_pit_stop_duration()
                    calculated_duration = pit_stop.calculate_pit_stop_duration()

                    # Mettre à jour le pilote
                    driver.tire_age = 0
                    driver.compound = pit_stop_data["compound"]
                    driver.next_pit_stop += 1

                    return calculated_duration
            return 0.0
        except ValueError as e:
            print(f"Error during pit stop for driver {driver.name}: {e}")
            return 0.0

    def get_driver_position(self, driver):
        """
        Calcule la position du pilote (uniquement parmi les pilotes 'alive').
        """
        sorted_drivers = sorted(
            [d for d in self.drivers_list if d.alive],
            key=lambda d: d.cumulative_lap_time
        )
        for idx, d in enumerate(sorted_drivers, start=1):
            if d == driver:
                driver.position = idx

    def _initialize_parameters(self):
        races_df = self.dataframes["races"]
        qualifyings_df = self.dataframes["qualifyings"]

        race_row = races_df[
            (races_df["season"] == self.season) & (races_df["location"] == self.gp_location)
        ]
        if race_row.empty:
            raise ValueError(
                f"No race found for location '{self.gp_location}' in season {self.season}."
            )
        self.race_id = race_row["id"].iloc[0]
        self.number_of_laps = race_row.iloc[0]["nolapsplanned"]

        # Grille de départ
        merged_data = qualifyings_df.merge(
            races_df, 
            left_on="race_id", 
            right_on="id", 
            suffixes=("_qualifying", "_race")
        )
        qualifying_rows = merged_data[
            (merged_data["season"] == self.season) & 
            (merged_data["location"] == self.gp_location)
        ]
        if qualifying_rows.empty:
            raise ValueError(
                f"No qualifying data found for '{self.gp_location}' in season {self.season}."
            )
        sorted_rows = qualifying_rows.sort_values(by="position")
        self.starting_grid = list(zip(sorted_rows["driver_id"], sorted_rows["position"]))

        # Créer les objets pilotes
        driver_ids = sorted_rows["driver_id"].unique().tolist()
        drivers_df = self.dataframes["drivers"]

        for driver_id in driver_ids:
            driver_row = drivers_df[drivers_df["id"] == driver_id]
            if driver_row.empty:
                continue
            driver_name = driver_row.iloc[0]["name"]
            driver_obj = Driver(
                season=self.season,
                race_id=self.race_id,
                dataframes=self.dataframes,
                name=driver_name
            )
            self.drivers_list.append(driver_obj)

        
    def simulate_dnf_lap(self, driver):
        driver.accident_dnf_lap = (
            np.random.randint(1, self.number_of_laps + 1)
            if np.random.binomial(1, driver.accident_dnf_probability)
            else None
        )
        driver.failure_dnf_lap = (
            np.random.randint(1, self.number_of_laps + 1)
            if np.random.binomial(1, driver.failure_dnf_probability)
            else None
        )
        potential_dnf_laps = [
            lap for lap in (driver.accident_dnf_lap, driver.failure_dnf_lap) if lap is not None
        ]
        driver.earliest_dnf_lap = min(potential_dnf_laps, default=None)


In [ ]:
if __name__ == "__main__":
    data_loader = DataLoader(db_path="F1_timingdata_2014_2019.sqlite")
    dataframes = data_loader.load_data()

    run_simulation = Run(season=2019, gp_location="SaoPaulo", dataframes=dataframes)
    run_simulation.run()

    # Affichez un résumé des résultats
    for driver in run_simulation.drivers_list:
        print(f"Driver: {driver.name}, Cumulative Lap Time: {driver.cumulative_lap_time}")
    display(run_simulation.laps_summary)

In [ ]:

if __name__ == "__main__":
    
    data_loader = DataLoader(db_path="F1_timingdata_2014_2019.sqlite")
    dataframes = data_loader.load_data()

    # Sélectionner une saison et un nom de GP
    season_test = 2019
    gp_location_test = "SaoPaulo"

    run_simulation = Run(
        season=season_test,
        gp_location=gp_location_test,
        dataframes=dataframes
    )

    for driver in run_simulation.drivers_list:
        run_simulation.simulate_dnf_lap(driver)
        print(f"Driver: {driver.name}")
        print(f"  Accident lap: {driver.accident_dnf_lap}")
        print(f"  Failure lap: {driver.failure_dnf_lap}")
        print(f"  First DNF lap: {driver.earliest_dnf_lap}")

    print("\n=== Fin du test ===")